# IEOR 4703 -- Monte Carlo Simulation for FE

### To calculate $ P(L>C) $ where $L=\sum Y_i e_i$ with $e_i$ exposure to $i^{th}$ credit and $Y_i$ is default indicator w/ correponding default probability of $p_i$

### $P(L > C) = E( 1_{\{L>C\}} )$

### We assume exposures, $e_i$, are independent

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import scipy as sp
from time import time

## Data

In [ ]:
# data
# exposures (in millions) and correponding default probabilities

exposures = np.array([5,       8,    9,   11,    3,    2,    2,    4,    7,    3,    4,    7,   15,   11,    4,    5])
defaultP  = np.array([0.10, 0.20, 0.10, 0.05, 0.15, 0.13, 0.10, 0.08, 0.06, 0.07, 0.05, 0.05, 0.02, 0.04, 0.09, 0.12])

totalExposure = np.sum(exposures)

expectedLoss_exact = np.sum(defaultP*exposures)

# threshold
C = 80

print("total exposure = ", totalExposure)
print("expectedLoss=", expectedLoss_exact)
print("threshold C = ", C)

total exposure =  100
expectedLoss= 7.66
threshold C =  80


## Standard/naive simulation (i.e. not utilizing tilted exponential)

### For the case that C is close to $E(L)$ should work well, however as $C$ gets much larger than $E(L)$ it wouldn't

In [ ]:
np.random.seed(1293561)

m = len(defaultP)
N = 100000
countP = 0

for i in range(N):

    U = np.random.rand(m) # U~Uniform(0,1)

    indicatorP = U<defaultP
    lossP = np.sum(indicatorP*exposures)
    if lossP>C:
        countP = countP+1


# P(L > C) = E( indicator(L>C) )
probP_L_greater_than_C = countP/N
print("P(L>C) =", probP_L_greater_than_C)

P(L>C) = 0.0


## Utilizing tilted exponential for large C, to assess it against standard/naive simulation
### we calculate both of them simulataneously
### For tilted exponetial, we need to find optimal value for hyperparameter $t$ for given $C$

In [ ]:
# we use tilted probability for the case that C >> E(L)
# there are many methods to find t, we use a proxy bisection method

# try different hyperparameters to assess
for t in [-1, 0.05, 0.1, 0.2, 0.28, 0.3, 0.33, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 5]:
    defaultQ = (defaultP*np.exp(t*exposures)) / (defaultP*np.exp(t*exposures)+(1-defaultP))
    C_tilde = np.sum(defaultQ*exposures)
    print ("C_tilde=", C_tilde, "for t=", t, " (threshold=", C,")")

C_tilde= 0.13393164779090747 for t= -1  (threshold= 80 )
C_tilde= 10.538263613263737 for t= 0.05  (threshold= 80 )
C_tilde= 14.600424127341277 for t= 0.1  (threshold= 80 )
C_tilde= 27.665937835568464 for t= 0.2  (threshold= 80 )
C_tilde= 42.38463050089311 for t= 0.28  (threshold= 80 )
C_tilde= 46.17614249167741 for t= 0.3  (threshold= 80 )
C_tilde= 51.62164233942069 for t= 0.33  (threshold= 80 )
C_tilde= 62.43981279119345 for t= 0.4  (threshold= 80 )
C_tilde= 73.15849766784686 for t= 0.5  (threshold= 80 )
C_tilde= 80.11255261585482 for t= 0.6  (threshold= 80 )
C_tilde= 84.90644589895808 for t= 0.7  (threshold= 80 )
C_tilde= 88.38158041053322 for t= 0.8  (threshold= 80 )
C_tilde= 90.98590687820433 for t= 0.9  (threshold= 80 )
C_tilde= 92.9751717364854 for t= 1  (threshold= 80 )
C_tilde= 99.9985579303754 for t= 5  (threshold= 80 )


In [ ]:
# from the above bisection
t = 0.60
defaultQ = (defaultP*np.exp(t*exposures)) / (defaultP*np.exp(t*exposures)+(1-defaultP))

In [ ]:
#np.random.seed(1293561)
np.random.seed(41433561)
m = len(defaultP)
N = 100000
countP = 0
countQ = 0


for i in range(N):

    U = np.random.rand(m) # U~Uniform(0,1)

    indicatorP = U<defaultP
    lossP = np.sum(indicatorP*exposures)

    if lossP>C:
        countP = countP+1


    indicatorQ = U<defaultQ
    lossQ = np.sum(indicatorQ*exposures)
    #
    # pay attention to the way likelihood is being calculated
    numerator   = np.prod(np.power(defaultP,indicatorQ)*np.power(1-defaultP,1-indicatorQ))
    denominator = np.prod(np.power(defaultQ,indicatorQ)*np.power(1-defaultQ,1-indicatorQ))
    likelihood = numerator/denominator
    #
    if lossQ>C:
        # pay attention to this part -- very important
        countQ = countQ + 1 * likelihood

# P(L > C) = E( indicator(L>C) )
probP_L_greater_than_C = countP/N
probQ_L_greater_than_C = countQ/N

print("P_P(L>C) =", probP_L_greater_than_C, ", P_Q(L>C) =", probQ_L_greater_than_C)

P_P(L>C) = 0.0 , P_Q(L>C) = 3.92960802795819e-11
